<a href="https://colab.research.google.com/github/Drmcoelho/APILLMML/blob/main/multimodal_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Etapa 0: Estrutura e Roadmap Interativo

> **Visão Geral:**  
> Nesta etapa inicial, definimos de forma detalhada toda a organização das etapas subsequentes. Use o sumário clicável para navegar e prepare-se para uma jornada de desenvolvimento multimodal robusta.

---

<details>
<summary><strong>📑 Sumário Interativo</strong></summary>

1. [Etapa 1: Montagem de Ambiente & Segurança](#etapa-1-montagem-de-ambiente--seguranca)  
2. [Etapa 2: Seleção e Ingestão de PDFs](#etapa-2-selecao-e-ingestao-de-pdfs)  
3. [Etapa 3: Extração Avançada de Texto](#etapa-3-extracao-avancada-de-texto)  
4. [Etapa 4: Análise Exploratória Clínica](#etapa-4-analise-exploratoria-clinica)  
5. [Etapa 5: Pré-processamento & Tokenização](#etapa-5-pre-processamento--tokenizacao)  
6. [Etapa 6: Construção de Dados Multimodal](#etapa-6-construcao-de-dados-multimodal)  
7. [Etapa 7: Modelos Candidatos & Hyperparams](#etapa-7-modelos-candidatos--hyperparams)  
8. [Etapa 8: Treinamento Avançado](#etapa-8-treinamento-avancado)  
9. [Etapa 9: Avaliação Clínica Especializada](#etapa-9-avaliacao-clinica-especializada)  
10. [Etapa 10: Exportação do Modelo Ótimo](#etapa-10-exportacao-do-modelo-otimo)  
11. [Etapa 11: Deploy & Cloud-Burst](#etapa-11-deploy--cloud-burst)  
12. [Etapa 12: Monitoramento & Atualização](#etapa-12-monitoramento--atualizacao)  
</details>

---

## 📋 Roadmap em 3 Níveis

| Fase           | Objetivo Principal                                          | Duração       |
|:--------------:|:------------------------------------------------------------|:-------------:|
| **Básico**     | PDF → Texto → Fine-tune inicial                             | 1–2 dias      |
| **Intermediário**| Imagens & Áudio → Fusion multimodal                       | 2 semanas     |
| **Avançado**   | Segurança, Compliance, Deploy, Monitoramento                | 1 semana      |

---

## 🔧 Cheatsheet Rápida

| Componente                | Comando/Referência                            | Notas Rápidas                                           |
|:-------------------------:|:-----------------------------------------------|:--------------------------------------------------------|
| Montar Drive              | `drive.mount('/content/drive')`                | Use `force_remount=True` para atualizar montagem        |
| Extração (PyMuPDF)        | `fitz.open(...).get_text("text")`              | Mais rápido que pdfminer                                |
| Tokenização (Clinical)    | `AutoTokenizer.from_pretrained(...)`           | Modelos Bio_ClinicalBERT, etc.                          |
| Mixed Precision           | `TrainingArguments(fp16=True, bf16=True)`      | Habilitar BF16 em Colab Pro                             |
| DVC Data/Model Versioning | `dvc run -n step ...`                          | Reprodutibilidade completa                              |
| PHI Detection             | `spaCy EntityRuler` + regex                    | Redirecionar logs para `/mnt/data/logs`                 |


# 🌟 Introdução Mega Robusta

Este notebook guia você em um **pipeline multimodal** de aprendizado profundo para **medicina**, desde a ingestão de PDFs até o **deploy** de um modelo clínico, com:

- **Cheatsheets** e **dicas de mestre** para acelerar o desenvolvimento.
- **Tabelas de referência** para consulta imediata.
- **Widgets interativos** e **visualizações avançadas**.
- **Segurança** e **compliance** (HIPAA/GDPR) embutidos.
- **Estratégia de cache** para reutilização de dados entre sessões.

Prepare-se para uma experiência de codificação **robusta**, **inovadora** e **produtiva**!


## Etapa 1: Montagem de Ambiente & Segurança

Nesta etapa, configuramos o ambiente **Google Colab Pro**, instalamos dependências com cache, configuramos `accelerate`, gerenciamos segredos via variáveis de ambiente nativas do Colab com fallback interativo, fazemos login no **Weights & Biases** automaticamente e criamos diretórios de cache.


In [ ]:
import os, json
import torch
from google.colab import drive
import wandb
from getpass import getpass
from IPython.display import display, HTML

# --- Parte de Configuração de Ambiente ---

# 1. Verificar GPU & Memória
print("⏳ Verificando recursos de hardware...")
if 'COLAB_GPU' not in os.environ or not torch.cuda.is_available():
    raise EnvironmentError("Por favor, selecione Runtime > Change runtime type > GPU e High-RAM.")
print(f"✅ GPU: {torch.cuda.get_device_name(0)}, Memória Aproximada: ~25GB")

# 2. Montar Google Drive
print("⏳ Montando Google Drive...")
try:
    # Tenta montar sem forçar a remontagem primeiro
    drive.mount('/content/drive', force_remount=False)
    print("✅ Google Drive montado ou já conectado.")
except Exception as e:
    print(f"⚠️ Falha na montagem inicial ({e}). Tentando forçar remontagem...")
    try:
        # Se falhar, tenta forçar a remontagem
        drive.mount('/content/drive', force_remount=True)
        print("✅ Google Drive remontado com sucesso.")
    except Exception as e_force:
        # Se a remontagem forçada também falhar, exibe um erro crítico
        print(f"❌ Falha crítica ao montar Google Drive: {e_force}")
        raise RuntimeError("Falha na montagem do Google Drive. A execução não pode continuar.")

# Definindo o caminho base para os dados no Drive
DATA_ROOT = '/content/drive/MyDrive/med-llm/data'
# Definindo o subdiretório para PDFs raw
PDF_ROOT = f"{DATA_ROOT}/raw/pdfs"
# Cria o diretório base de dados se ele não existir
os.makedirs(DATA_ROOT, exist_ok=True)
print("📂 Caminho raiz dos dados (DATA_ROOT):", DATA_ROOT)

# 3. Criar Diretórios de Cache
# Criando diretórios para armazenar dados intermediários e modelos.
# Utilizar /content para cache local pode ser mais rápido que no Drive para arquivos temporários,
# mas pode não persistir entre sessões se a instância do Colab for reciclada.
# Os diretórios em /mnt/data são frequentemente usados por bibliotecas como accelerate.
dirs_to_create = ['/content/data/pdf_texts', '/content/data/datasets', '/content/data/models']
# Garante que o diretório pai /content/data existe antes de criar os subdiretórios
os.makedirs('/content/data', exist_ok=True)
for d in dirs_to_create:
    os.makedirs(d, exist_ok=True)
print("✅ Diretórios de cache locais criados/verificados:", dirs_to_create)
# Nota: Se você precisa persistência do cache entre sessões, considere usar o Drive.

# --- Parte de Gerenciamento de Segredos ---

# Caminho para o arquivo JSON que armazenará os segredos no Google Drive
SECRETS_PATH = '/content/drive/MyDrive/secrets.json'

def load_secrets(path):
    """Carrega segredos de um arquivo JSON."""
    try:
        with open(path, 'r') as f:
            return json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        # Retorna um dicionário vazio se o arquivo não existir ou for inválido
        return {}

def save_secrets(secrets, path):
    """Salva segredos em um arquivo JSON com permissões restritas."""
    # Abre o arquivo com permissões de leitura/escrita apenas para o proprietário (0o600)
    # O_WRONLY: Abre para escrita apenas
    # O_CREAT: Cria o arquivo se não existir
    # O_TRUNC: Trunca o arquivo para comprimento zero se ele existir
    # 0o600: Permissões octais (rw-------)
    fd = os.open(path, os.O_WRONLY | os.O_CREAT | os.O_TRUNC, 0o600)
    with os.fdopen(fd, 'w') as f:
        json.dump(secrets, f, indent=4) # Use indent para formatação legível

# Carrega os segredos existentes do arquivo no Drive
secrets = load_secrets(SECRETS_PATH)
required_keys = ['HF_TOKEN', 'WANDB_API_KEY', 'UMLS_API_KEY']

# Solicita as chaves ausentes ou vazias interativamente
print("🔑 Verificando e configurando segredos...")
for key in required_keys:
    # Verifica se a chave não existe ou se o valor é vazio
    if not secrets.get(key):
        print(f"🤔 O segredo '{key}' não foi encontrado ou está vazio no arquivo {SECRETS_PATH}.")
        # Solicita a chave usando getpass para entrada segura (não exibe o texto digitado)
        secrets[key] = getpass(f'👉 Digite o valor para "{key}": ').strip()

# Salva os segredos atualizados de volta no arquivo JSON no Drive
save_secrets(secrets, SECRETS_PATH)
print(f"✅ Segredos salvos/atualizados em {SECRETS_PATH}.")

# Define as variáveis de ambiente com os segredos carregados/solicitados
for key, value in secrets.items():
    os.environ[key] = value
print("✅ Variáveis de ambiente para segredos configuradas.")

# --- Parte de Autenticação e Configuração de Ferramentas ---

# 4. Login Weights & Biases (Agora que WANDB_API_KEY deve estar no ambiente)
# Tenta autenticar com a chave W&B carregada/solicitada
wandb_api_key = os.environ.get('WANDB_API_KEY')
if wandb_api_key and len(wandb_api_key) >= 40: # Verifica se a chave parece válida (W&B API keys geralmente têm 40 caracteres, mas a validação completa é feita pelo wandb.login)
    try:
        print("⏳ Tentando autenticar no Weights & Biases...")
        wandb.login(key=wandb_api_key, relogin=True) # relogin=True pode ajudar em alguns ambientes
        print("✅ Weights & Biases autenticado com sucesso.")
    except Exception as e:
        print(f"❌ Falha ao autenticar Weights & Biases com a chave fornecida: {e}")
        print("⚠️ Verifique sua WANDB_API_KEY.")
else:
    print("⚠️ WANDB_API_KEY não encontrado ou inválido. Pulando autenticação W&B.")


# 5. Instalar Dependências com Cache
print("📦 Instalando dependências...")
# Lista de bibliotecas essenciais
dependencies = "pymupdf transformers datasets accelerate bitsandbytes wandb"
# Instala as dependências, usando um diretório de cache temporário para acelerar
os.system(f"pip install --quiet --cache-dir=/tmp/pip-cache {dependencies}")
print(f"✅ Dependências instaladas: {dependencies}")

# 6. Configurar accelerate
print("⚙ Configurando accelerate...")
# Executa a configuração padrão do accelerate (interativamente, mas 'yes' responde sim para todas as perguntas)
# Redireciona a saída para /dev/null para evitar poluir a saída do notebook
os.system("yes '' | accelerate config default > /dev/null 2>&1")

# Modifica o arquivo de configuração do accelerate para habilitar mixed precision e aumentar accumulation steps
cfg_path = 'accelerate_config.yaml'
if os.path.exists(cfg_path):
    try:
        with open(cfg_path, 'r') as f:
            config_content = f.read()

        # Faz as substituições para habilitar fp16 e bf16 e aumentar gradient_accumulation_steps
        config_content = config_content.replace('fp16: false', 'fp16: true')
        config_content = config_content.replace('bf16: false', 'bf16: true')
        config_content = config_content.replace('gradient_accumulation_steps: 1', 'gradient_accumulation_steps: 4')

        with open(cfg_path, 'w') as f:
            f.write(config_content)
        print("⚙ accelerate configurado para fp16/bf16 & gradient_accumulation_steps=4")

    except Exception as e:
        print(f"❌ Falha ao modificar o arquivo accelerate_config.yaml: {e}")
else:
    print(f"⚠️ Arquivo de configuração do accelerate '{cfg_path}' não encontrado após a execução do comando 'accelerate config default'.")


# Opcional: Gerar link para o notebook atualizado (útil se estiver usando o Colab)
try:
    from google.colab import output
    # Obtém o URL do notebook atual via JavaScript no ambiente Colab
    notebook_link = output.eval_js("window.location.href")
    # Exibe o link como HTML clicável
    display(HTML(f'<a href="{notebook_link}" target="_blank">🔗 Abrir este notebook em uma nova aba</a>'))
except Exception as e:
    print(f"⚠️ Não foi possível gerar o link do notebook: {e}")

print("\n✅ Configuração de ambiente e gerenciamento de segredos concluídos!")

⏳ Verificando recursos de hardware...
✅ GPU: Tesla T4, Memória Aproximada: ~25GB
⏳ Montando Google Drive...
Mounted at /content/drive
✅ Google Drive montado ou já conectado.
📂 Caminho raiz dos dados (DATA_ROOT): /content/drive/MyDrive/med-llm/data
✅ Diretórios de cache locais criados/verificados: ['/content/data/pdf_texts', '/content/data/datasets', '/content/data/models']
🔑 Verificando e configurando segredos...
🤔 O segredo 'UMLS_API_KEY' não foi encontrado ou está vazio no arquivo /content/drive/MyDrive/secrets.json.
👉 Digite o valor para "UMLS_API_KEY": ··········
✅ Segredos salvos/atualizados em /content/drive/MyDrive/secrets.json.
✅ Variáveis de ambiente para segredos configuradas.
⏳ Tentando autenticar no Weights & Biases...


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: drmatheuscoelho to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


✅ Weights & Biases autenticado com sucesso.
📦 Instalando dependências...
✅ Dependências instaladas: pymupdf transformers datasets accelerate bitsandbytes wandb
⚙ Configurando accelerate...
⚠️ Arquivo de configuração do accelerate 'accelerate_config.yaml' não encontrado após a execução do comando 'accelerate config default'.



✅ Configuração de ambiente e gerenciamento de segredos concluídos!
